## Calculate CRN Concentrations
<br>
  
#### Syntax
`CalcPars = crnLoopFunction(RunPars, CosmoPars, CalcPars)` <br>
#### Input 
`RunPars` : dictionary of parameters relevant for scenario runs. <br>
`CosmoPars` : dictionary of parameters relevant for cosmo calculations. <br>
`CalcPars` : dictionary of parameters for the calculatiosn <br>
#### Variables Used
##### RunPars
`s` : number of scenarios <br>
`RockfallMat` : rockfall erosion matrix of size [s,total_time] <br>
`ERs` : the true erosion rates (L / total_time) (cm yr-1) <br>
`max_depths` : maximum solve depths for He and CRN (cm) <br>
`total_time` : total time in the model run (yrs) <br>

##### Cosmo Pars
`CRNz` : depth vector of len(max_depths / dz) <br>
`dz` : node spacing for CRN depth vectors (cm) <br>
#### Output
##### Calc Pars
`C14 Depth Vectors ' + str(i)` : matrices of size [len(CRNz),total_time] containing depth C14 profiles (at g-1) <br>
`'Be10 Depth Vectors ' + str(i)` : matrices of size [len(CRNz),total_time] containing depth Be10 profiles (at g-1) <br>
`CRN Surface Conc.` = surface concentrations of size [s,total_time] containing surface (to 6cm) concentrations (at g-1) <br>

#### Notes
**Date of Creation:** 1. Juli 2021 <br>
**Author:** Donovan Dennis <br>
**Update:** <br>
04.03.2022 : Renamed to `crnLoopFunction` from `crnLoopFunctionmtest` for publication.

In [5]:
def crnLoopFunction(RunPars, CosmoPars, CalcPars):
    # import the important parameters
    s = RunPars['scenarios']
    RockfallMat = RunPars['RockfallMatrix']
    ERs = RunPars['TrueErosionRates']
    max_depths = RunPars['max_depths']
    total_time = RunPars['total_time']
    dz = RunPars['dz']
    CRNz = CosmoPars['CRNz']
    nz = CosmoPars['CRNnz']

    # matrix for the surface concentrations of 3 x time x scenarios
    AllScenariosSurf = np.empty((3, total_time, s))
    
    # Loop through all of the scenarios
    for i in range(s):
        BeSpall = np.e**(-CosmoPars['mu'] * CosmoPars['CRNz']) * \
        ((CosmoPars['SLHL_Be10']) / (CosmoPars['Be10 Decay'] + CosmoPars['mu'] * ERs[i]))
        
        BeMuons = np.e**(-CosmoPars['rock_rho'] / CosmoPars['L3 Be'] * CRNz) * \
        (CosmoPars['B_Be'] / (CosmoPars['Be10 Decay'] + (CosmoPars['rock_rho'] / CosmoPars['L3 Be']) *ERs[i]))  

        CSpall = np.e**(-CosmoPars['mu'] * CosmoPars['CRNz']) * \
        ((CosmoPars['SLHL_C14']) / (CosmoPars['C14 decay'] + CosmoPars['mu'] * ERs[i]))
        
        CMuons1 = np.e**(-CosmoPars['rock_rho'] / CosmoPars['L1 C14'] * CRNz) * \
        (CosmoPars['C14 f_tot'] * CosmoPars['C14 A1'] / (CosmoPars['C14 decay'] + (CosmoPars['rock_rho'] / CosmoPars['L1 C14']) * ERs[i]))
        
        CMuons2 = np.e**(-CosmoPars['rock_rho'] / CosmoPars['L2 C14'] * CRNz) * \
        (CosmoPars['C14 f_tot'] * CosmoPars['C14 A2'] / (CosmoPars['C14 decay'] + (CosmoPars['rock_rho'] / CosmoPars['L2 C14']) * ERs[i]))  
        
        conc14C = CSpall + CMuons1 + CMuons2 # an empty vector for the annual C14 concentrations
        conc10Be = BeSpall + BeMuons# empty vector for the annual Be10 concentrations
        muon10Be = BeMuons
        spall10Be = BeSpall
        # empty matrix of size max depths CRN x time; will store annual depth profiles for C14.
        C14DepthMatrix = np.empty((nz,total_time))
        # empty matrix of size max depths CRN x time; will store annual depth profiles for Be10.
        Be10DepthMatrix = np.empty((nz,total_time))            
        # empty matrix for just the surface concentrations
        crnSurf = np.empty((3,total_time))
        
        CRNz = CosmoPars['CRNz']  # import depth vector to solve at

        
        for j in range(0,total_time):
            # what is the erosion event depth for this scenario, this year (cm)
            erode_z = RockfallMat[i,j] 
            # calculate the depth profiles of Be10 and C14 after eroding
            conc10Be, muon10Be, spall10Be, conc14C = crnErodeFxn(conc10Be, muon10Be, spall10Be, conc14C, erode_z, CosmoPars)
            # calculate surface concentrations for a sample that is 5 cm thick
            crnSurf[0,j] = np.average(conc14C[0:int(6/dz)])
            crnSurf[1,j] = np.average(conc10Be[0:int(6/dz)])
            # calculate the surface 14C/10Be ratio
            crnSurf[2,j] = crnSurf[0,j] / crnSurf[1,j]
            # assign depth profilesto export matrix
            C14DepthMatrix[:,j] = conc14C
            Be10DepthMatrix[:,j] = conc10Be
            #Be10MDepthMatrix[:,j] = muon10Be
            #Be10SDepthMatrix[:,j] = spall10Be
            
        AllScenariosSurf[:,:,i] = crnSurf
        
        # save to dictionary
        CalcPars['C14 Depth Vectors ' + str(i)] = C14DepthMatrix
        CalcPars['Be10 Depth Vectors ' + str(i)] = Be10DepthMatrix
        #CalcPars['Be10 Mu Depth Vectors ' + str(i)] = Be10MDepthMatrix
        #CalcPars['Be10 Sp Depth Vectors ' + str(i)] = Be10SDepthMatrix

    # save to dictionary
    CalcPars['CRN Surface Conc.'] = AllScenariosSurf
            
    return CalcPars